In [ ]:
# Experiment 2 - 1 output layer with 1 loss function - mse. and do hyper parameter tuning.
from utils.data_preprocess import load_data, split_xy_save, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)


single_leakage, two_leakage = load_data(total_samples = cfg['experiment']['total_samples'])
two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
data = data.drop(columns=["mfc10_residual"], inplace=False)

# why on earth are we scaling based on geometry rather than standard deviation and mean
# data['x2'] = data['x2'] / cfg['geometry']['span']
# data['x1'] = data['x1'] / cfg['geometry']['span']

# data['y2'] = data['y2'] / cfg['geometry']['width']
# data['y1'] = data['y1'] / cfg['geometry']['width']

data['x2'] = data['x2'].replace(np.nan, -5)
data['y2'] = data['y2'].replace(np.nan, -5)
# or try generating a random number

y = data[['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2']]
x = data.drop(['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

In [ ]:
# Fitting a simple linear regression model
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

# converting the predictions to certainity of 0 and 1 (May not be a good idea)
y_pred[:,-2][np.abs(y_pred[:,-2]) < 0.5] = 0
y_pred[:,-2][np.abs(y_pred[:,-2]) > 0.5] = 1
y_pred[:,-1][np.abs(y_pred[:,-1]) < 0.5] = 0
y_pred[:,-1][np.abs(y_pred[:,-1]) > 0.5] = 1

y_pred = y_pred.transpose()

mse_x1 = mean_squared_error(y_test['x1'], y_pred[0], squared=True)
mse_x2 = mean_squared_error(y_test['x2'], y_pred[2], squared=True)

mse_y1 = mean_squared_error(y_test['y1'], y_pred[1], squared=True)
mse_y2 = mean_squared_error(y_test['y2'], y_pred[3], squared=True)

mse_leak1 = mean_squared_error(y_test['leak_1'], y_pred[4], squared=True)
mse_leak2 = mean_squared_error(y_test['leak_2'], y_pred[5], squared=True)

mse = [mse_x1, mse_x2, mse_y1, mse_y2, mse_leak1, mse_leak2]

mse

In [ ]:
y_test